# Spatial Correlation: calculating Bivariate Moran's I and Local Moran's I with census data retrieved from CENSUS API

This notebook demonstrates how to calculate Bivariate Moran's I and Local Moran's I based on census data. To be specific, it employs census tract level Social Vulnerability Index (SVI) data, focusing on Chicago, Illinois. 

### Data: 
* Census Tracts: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html <br>
* Social Vulnerability Index (SVI): https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2020.html

Social Vulnerability Index (SVI) is a measure of the social vulnerability of a community. It is calculated based on 16 variables, including poverty, lack of vehicle access, and crowded housing. The SVI is calculated at the census tract level, with a higher value indicating a higher level of social vulnerability. The SVI data used in this notebook is from the Centers for Disease Control and Prevention (CDC) and is based on the 2020 SVI dataset.

![](https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI-Variables.png?_=02699)

Source: https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2020.html


### Steps:
1. Retrieve census data
2. Calculate non-spatial correlation
3. Calculate spatial correlation (Bivariate Moran's I & Bivariate Local Moran's I)
4. K-means clustering


# Import Packages

A Python package is a way of organizing related Python modules into a single directory hierarchy. It provides a mechanism for grouping Python code files, resources, and configuration settings in a structured manner, making it easier to manage and distribute code. They also facilitate code reuse and distribution by allowing developers to bundle related functionality together and share it with others.

The following packages are used in this notebook:<br>
`requests` is a Python package to send HTTP requests using Python. It allows you to send HTTP requests and get responses from the web. <br>
source: https://requests.readthedocs.io/en/latest/ <br>

`libpysal` is a Python package for spatial analysis. It provides foundational algorithms and data structures that support the rest of the library. <br>
source: https://pysal.org/libpysal/ <br>

`esda` is a Python package and implements methods for the analysis of both global (map-wide) and local (focal) spatial autocorrelation, for both continuous and binary data. In addition, the package increasingly offers cutting-edge statistics about boundary strength and measures of aggregation error in statistical analyses. <br>
source: https://pysal.org/esda/index.html <br>

In [ ]:
# New Packages
import requests
import libpysal
import esda
from scipy.stats import pearsonr

# Packages that investigated in the previous project
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Etc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Census tracts of Chicago, IL
tract_geom = gpd.read_file('./data/tracts_Cook_IL.geojson')
tract_geom

In [ ]:
# Check the boundary of the tracts
tract_geom.plot()

# 1. Retrieve Census Data

The American Community Survey (ACS) is an ongoing survey that provides data every year -- giving communities the current information they need to plan investments and services. The ACS covers a broad range of topics about social, economic, demographic, and housing characteristics of the U.S. population.

*HOWEVER*, manually downloading the data from the ACS website is tedious. The Census Bureau provides an API (Application Programming Interface) to access the data. The API is a way to access the data in a structured way. The API allows you to request data from the Census Bureau and receive it in a format that is easy to work with.

https://www.census.gov/data/developers/data-sets/acs-5year.2020.html#list-tab-1806015614

To use API, you need to request an API key from the Census Bureau. The API key is a unique identifier that is used to authenticate requests associated with your project for usage and billing purposes.

https://api.census.gov/data/key_signup.html

After you submit the request, you will receive an email with the API key. Check your email and copy the API key.

In [ ]:
table_name = 'S1701_C01_040E' # ALL INDIVIDUALS WITH INCOME BELOW 150% OF POVERTY LEVEL
state = '17'
county = '031'
API_KEY = 'YOUR API KEY' # Paste your API key here 

api_address = f'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,{table_name}&for=tract:*&in=state:{state}&in=county:{county}&key={API_KEY}'
api_address

In [ ]:
# if you see 200 as the output, it means the request was successful
requests.get(api_address)

In [ ]:
# Convert the response to a JSON object
# The output will be a nested list. The first element of the list is the header and the rest are the data
response = requests.get(api_address).json()
response

In [ ]:
# Slice the first list, which will become the header of the DataFrame
response[0]

In [ ]:
# Slice the rest of the lists, which will become the data of the DataFrame
response[1:]

In [ ]:
# Create a DataFrame from the response of API request
results = pd.DataFrame(response[1:], columns=response[0])
results

In [ ]:
# Aggretate State, County, and Tract codes to create GEOID
results['GEOID'] = results['state'] + results['county'] + results['tract']
results

In [ ]:
# Join the results with the tract geometries
tract_pov = tract_geom.merge(results, on='GEOID')
tract_pov

In [ ]:
# Rename the column name (S1701_C01_040E) to a more meaningful name (Poverty_count)
tract_pov = tract_pov.rename(columns={'S1701_C01_040E': 'Poverty_count'})
tract_pov

In [ ]:
# Plot the poverty count, but the result will be problematic due to the data type
tract_pov.plot('Poverty_count', figsize=(6, 6))

In [ ]:
# Check the statistics of the column 
tract_pov['Poverty_count'].describe()

In [ ]:
# Check the data type of the columns in the DataFrame
tract_pov.dtypes

In [ ]:
# Convert the data type of the column to integer
tract_pov['Poverty_count'] = tract_pov['Poverty_count'].astype(int)

# Check the data type of the columns in the DataFrame
tract_pov.dtypes

In [ ]:
# Plot the poverty count, but still the result is misleading since the data is not normalized per population in each tract
tract_pov.plot('Poverty_count', legend=True,  figsize=(6, 6))

In [ ]:
# Calculate the poverty ratio by dividing the poverty count by the population
tract_pov['Poverty_ratio'] = tract_pov['Poverty_count'] / tract_pov['Pop'] # Some tracts have no population, resulting NaN in the ratio
tract_pov['Poverty_ratio'] = tract_pov['Poverty_ratio'].fillna(0) # Replace NaN with 0
tract_pov

In [ ]:
# Check the statistics of the column 
tract_pov['Poverty_ratio'].describe()

In [ ]:
# Plot the poverty ratio
tract_pov.plot('Poverty_ratio', legend=True, figsize=(6, 6))

---
### *Exercise*
1. (3 points) Retrive Census data with the following parameters:
* Table Name: 'S2701_C05_001E' # Percent of people without health insurance
* State: '17' # Illinois
* County: '031' # Cook County
* API KEY: 'Your API Key' 
        

```python
table_name = '' # Table Name as a string
state = '' # State ID as a string
county = '' # County ID as a string
API_KEY = '' # Your API Key as a string

api_address_text = f'https://api.census.gov/data/2020/acs/acs5/subject?get={table_name}&for=tract:*&in=state:{state}&in=county:{county}&key={API_KEY}'
response_list = requests.get(api_address_text).json()
response_list
```

---

In [ ]:
# Your code here
table_name = '' # Table Name as a string
state = '' # State ID as a string
county = '' # County ID as a string
API_KEY = '' # Your API Key as a string

api_address_text = f'https://api.census.gov/data/2020/acs/acs5/subject?get={table_name}&for=tract:*&in=state:{state}&in=county:{county}&key={API_KEY}'
response_list = requests.get(api_address_text).json()
response_list

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert table_name == 'S2701_C05_001E'
assert state == '17'
assert county == '031'
# assert API_KEY != '5ad4c26135eaa6a049525767607eecd39e19d237'
assert type(response_list) == list
assert len(response_list) == 1333
assert response_list[0] == ['S2701_C05_001E', 'state', 'county', 'tract']

print("Success!")

---
### *Exercise*

2. (3 points) Convert the response to a pandas DataFrame. <br>
2.1. Assign the first list of the nested list as the column names. <br>
2.2. Assign the rest of the nested list as the data. <br>

3. (3 points) Create a new column ('GEOID') by combining 'state', 'county', and 'tract' columns. <br>

```python
# Convert the response to a pandas DataFrame
results_df = pd.DataFrame(data=`Slice of the rest of the nested list`, columns=`Slice of the first list of the nested list`)

# Create a new column 'GEOID' by combining 'state', 'county', and 'tract' columns
results_df['GEOID'] = `Combine 'state', 'county', and 'tract' columns of the 'results_df' DataFrame` 
results_df
```

---

In [ ]:
# Your code here
# Convert the response to a pandas DataFrame
results_df = pd.DataFrame(data=`Slice of the nested list`, columns=`Slice of the nested list`)

# Create a new column 'GEOID' by combining 'state', 'county', and 'tract' columns
results_df['GEOID'] = `Add Columns`
results_df

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert type(results_df) == pd.DataFrame
assert results_df.shape[0] == 1332
assert 'S2701_C05_001E' in results_df.columns
assert 'GEOID' in results_df.columns

print("Success!")

---
### *Exercise*

4. (3 points) Join `tract_geom` GeoDataFrame and `results_df` DataFrame based on `GEOID` column, and save the result as `tract_ins`. <br>


``` python
tract_ins = `DATAFRAME A`.merge(`DATAFRAME B`, on=`COLUMN NAME`)
```

5. (3 points) Rename `S2701_C05_001E` column as `No_HS_pct`. <br>

``` python
tract_ins = tract_ins.rename(columns={`OLD COLUMN NAME`: `NEW COLUMN NAME`})
```

6. (3 points) Change the data type of `No_HS_pct` column to float. <br>

``` python
`DATAFRAME`[`COLUMN NAME`] = `DATAFRAME`[`COLUMN NAME`].astype(`DATA TYPE`)
```

---

In [ ]:
# Your code here

# Join `tract_geom` GeoDataFrame and `results_df` DataFrame based on `GEOID` column
tract_ins = `DATAFRAME A`.merge(`DATAFRAME B`, on=`COLUMN NAME`)

# Rename `S2701_C05_001E` column as `No_HS_pct`
tract_ins = tract_ins.rename(columns={`OLD COLUMN NAME`: `NEW COLUMN NAME`})

# Change the data type of `No_HS_pct` column to float
`DATAFRAME`[`COLUMN NAME`] = `DATAFRAME`[`COLUMN NAME`].astype(`DATA TYPE`)

tract_ins

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert 'S2701_C05_001E' not in tract_ins.columns
assert 'No_HS_pct' in tract_ins.columns
assert tract_ins['No_HS_pct'].dtypes == float

print("Success!")

# 2. Calculate Correlation Coefficient between Poverty and No Health Insurance

In [ ]:
# Replace Null value (-666666666.0) with 0 in the 'No_HS_pct' column
tract_ins['No_HS_pct'].replace(-666666666.0, 0, inplace=True)

# Merge the 'tract_ins' and 'tract_pov' DataFrames
tract_gdf = tract_geom.merge(tract_ins[['GEOID', 'No_HS_pct']], on='GEOID')
tract_gdf = tract_gdf.merge(tract_pov[['GEOID', 'Poverty_ratio']], on='GEOID')
tract_gdf

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 10))

# Plot the poverty ratio
tract_gdf.plot('Poverty_ratio', ax=axes[0], legend=True, scheme='NaturalBreaks', cmap='Reds')

# Plot the percentage of people without health insurance
tract_gdf.plot('No_HS_pct', ax=axes[1], legend=True, scheme='NaturalBreaks', cmap='Reds')

axes[0].set_title('Poverty Ratio')
axes[1].set_title('No Health Insurance')

for ax in axes:
    ax.set_axis_off()


In [ ]:
coef, pval = pearsonr(tract_gdf['Poverty_ratio'], tract_gdf['No_HS_pct'])

print(f"Pearson's correlation coefficient: {coef:.2f}")
print(f"P-value: {pval:.2f}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.scatter(tract_gdf['Poverty_ratio'], tract_gdf['No_HS_pct'], s=5, color='black', alpha=0.5)
ax.set_xlabel('Poverty Ratio')
ax.set_ylabel('No Health Insurance')

x = tract_gdf['Poverty_ratio'].values
y = tract_gdf['No_HS_pct'].values

ax.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)), color='red')
plt.show()

# 3. Caculate Spatial Correlation (Bivariate Moran's I and Bivariate Local Moran's I)

Global **Bivariate Moran's I** demonstrates how geographical phenomena are correlated over space, meaning whether closer things is more related than distant things. The method provides an index with the range -1 to 1; namely, -1 is a strong negative spatial autocorrelation and 1 is a strong positive spatial autocorrelation.
<br><br>
While Global Bivariate Moran's I only provides one index to demonstrate spatial autocorrelation, **Bivariate Local Indicator of Spatial Association (LISA)**, as known as Bivariate Local Moran's I explains where high (i.e., HH Cluster) and low (LL Cluster) values are clustered.

The following figure will demonstrate what **contiguity** means. Here, we use Queen's case. 

<div>
<img src="./image/contiguity.jpg" width="500"/>
</div>

In [ ]:
# Calculate spatial relationship (contiguity) of geometry
w = libpysal.weights.Queen.from_dataframe(tract_gdf) 

w.neighbors

Syntax for Bivariate Moran's I and Bivariate Local Moran's I are as follows:

```python
# Bivariate Moran's I
bv_mi = esda.moran.Moran_BV(`VARIABLE 1`, `VARIABLE 2`, `SPATIAL WEIGHTS MATRIX`)

# Bivariate Local Moran's I
bv_lm = esda.moran.Moran_Local_BV(`VARIABLE 1`, `VARIABLE 2`, `SPATIAL WEIGHTS MATRIX`)
```

bv_mi has the following attributes:
* `I` : Bivariate Moran's I value
* `p_sim` : p-value <br>

source: https://pysal.org/esda/generated/esda.Moran_BV.html#esda.Moran_BV

bv_lm has the following attributes:
* `Is` : Bivariate Local Moran's I value
* `q` : Quadrant (HH, HL, LH, LL)
* `p_sim` : p-value

source: https://pysal.org/esda/generated/esda.Moran_Local_BV.html#esda.Moran_Local_BV

In [ ]:
# CAclulate Bivariate Moran's I
bv_mi = esda.Moran_BV(tract_gdf['Poverty_ratio'], tract_gdf['No_HS_pct'], w)

In [ ]:
# Moran's I value
bv_mi.I

In [ ]:
# Moran's I p-value (significance level)
bv_mi.p_sim

In [ ]:
# Calculate Local Bivariate Moran's I
bv_lm = esda.Moran_Local_BV(tract_gdf['Poverty_ratio'], tract_gdf['No_HS_pct'], w, seed=17)

In [ ]:
# Classification of Bivariate Local Moran's I
# 1: High-High, 2: Low-High, 3: Low-Low, 4: High-Low
bv_lm.q

In [ ]:
# P-value of Bivariate Local Moran's I
bv_lm.p_sim

Let's plot Bivariate Local Moran's I result. 

In [ ]:
# Add the results of Bivariate Local Moran's I to the GeoDataFrame
tract_gdf['BV_LM'] = bv_lm.q
tract_gdf['BV_LM_pval'] = bv_lm.p_sim
tract_gdf

In [ ]:
# Enter results of Bivariate LISA into each census region based on the p-value
lm_dict = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}
for idx in range(tract_gdf.shape[0]):
    if bv_lm.p_sim[idx] < 0.05:
        tract_gdf.loc[idx, f'LISA_CLS'] = lm_dict[bv_lm.q[idx]]
    else:
        tract_gdf.loc[idx, f'LISA_CLS'] = 'NA'

tract_gdf

In [ ]:
# Check the number of each classification
tract_gdf['LISA_CLS'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

lisa_color = {'HH': '#FF0000', # Red
              'LH': '#FFC0CB', # Pink
              'LL': '#0000FF', # Blue
              'HL': '#87CEEB', # Skyblue
              'NA': '#f0f0f0'  # Light grey
             } 

tract_gdf.loc[tract_gdf['LISA_CLS'] == 'NA'].plot(ax=ax, color=lisa_color['NA'])
tract_gdf.loc[tract_gdf['LISA_CLS'] == 'HH'].plot(ax=ax, color=lisa_color['HH'])
tract_gdf.loc[tract_gdf['LISA_CLS'] == 'LH'].plot(ax=ax, color=lisa_color['LH'])
tract_gdf.loc[tract_gdf['LISA_CLS'] == 'LL'].plot(ax=ax, color=lisa_color['LL'])
tract_gdf.loc[tract_gdf['LISA_CLS'] == 'HL'].plot(ax=ax, color=lisa_color['HL'])

ax.set_axis_off()
plt.show()

In [ ]:
# Simplify the code using a for loop
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for key,val in lisa_color.items():
    tract_gdf.loc[tract_gdf['LISA_CLS'] == key].plot(ax=ax, color=val)

ax.set_axis_off()
plt.show()

# 4. Calculate Spatial Correlation for SVI data

In [ ]:
# Load the Social Vulnerability Index (SVI) data of Cook County, IL
tract_svi = gpd.read_file('./data/SVI_Cook_IL.geojson')

# Replace Null values (-999) with 0
tract_svi = tract_svi.replace(-999, 0)

tract_svi

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 10))
axes = axes.flatten()

for idx, col in enumerate(['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']):
    tract_svi.plot(col, ax=axes[idx], scheme='NaturalBreaks', cmap='Reds')
    axes[idx].set_title(col)

for ax in axes:
    ax.set_axis_off()
    
plt.show()

---
### *Exercise*

7. (3 points) Calculate conventional correlation (Pearson's r) between `RPL_THEME1` and `RPL_THEME3` columns in `tract_svi` GeoDataFrame. <br>

```python
   r_coef, p_value = pearsonr(`DATAFRAME`[`COLUMN 1`], `DATAFRAME`[`COLUMN 2`])
```

---

In [ ]:
# Your code here

r_coef, p_value = pearsonr(`DATAFRAME`[`COLUMN 1`], `DATAFRAME`[`COLUMN 2`])

print(f"Pearson's correlation coefficient: {r_coef:.2f}, P-value: {p_value:.2f}")

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert round(r_coef, 2) == 0.83
assert p_value == 0.0

print("Success!")

---
### *Exercise*

8. (3 points) Calculate contiguity between census tracts in `tract_svi` GeoDataFrame <br>

```python
   w_svi = libpysal.weights.Queen.from_dataframe(`GeoDataFrame`)
```

---

In [ ]:
# Your code here
w_svi = libpysal.weights.Queen.from_dataframe(`GeoDataFrame`)

w_svi.neighbors


In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert len(w_svi.neighbors) == 1331 
assert w_svi.neighbors[0] == [1169, 659, 820, 662, 663, 716, 766]

print("Success!")

---
### *Exercise*

9. (3 points) Calculate Bivariate Moran's I and Bivariate Local Moran's I by entering appropriate values in the following syntax. <br>You want to calculate the spatial correlation between `RPL_THEME1` and `RPL_THEME3`.  <br>


```python
   # Bivariate Moran's I
   bv_mi_svi = esda.moran.Moran_BV(`VARIABLE 1`, `VARIABLE 2`, `SPATIAL WEIGHTS MATRIX`)

   # Bivariate Local Moran's I
   bv_lm_svi = esda.moran.Moran_Local_BV(`VARIABLE 1`, `VARIABLE 2`, `SPATIAL WEIGHTS MATRIX`, seed=17)
```

**Note**: seed is used to ensure the reproducibility of the result. Please keep the value as 17. 

---

In [ ]:
# Your code here

# Bivariate Moran's I
bv_mi_svi = esda.moran.Moran_BV(`VARIABLE 1`, `VARIABLE 2`, `SPATIAL WEIGHTS MATRIX`)

# Bivariate Local Moran's I
bv_lm_svi = esda.moran.Moran_Local_BV(`VARIABLE 1`, `VARIABLE 2`, `SPATIAL WEIGHTS MATRIX`, seed=17)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert round(bv_mi_svi.I, 3) == 0.688
assert round(bv_mi_svi.p_sim, 3) == 0.001
assert list(bv_lm_svi.q[0:5]) == [1, 1, 1, 1, 3]
assert list(bv_lm_svi.p_sim[0:5]) == [0.001, 0.003, 0.164, 0.302, 0.011]

print("Success!")

---
### *Exercise*

10. (3 points) Examine the following website to understand the meaning of the Bivariate Local Moran's I result. <br>
Then, assign the appropriate results of quadrant and significant level (p-value) to `bv_lm_svi` and `bv_lm_svi_pval` in `tract_svi` GeoDataFrame, respectively. <br>

website: https://pysal.org/esda/generated/esda.Moran_Local_BV.html#esda.Moran_Local_BV

```python
   tract_svi['bv_lm_svi'] = `Bivariate Local Morans I result`
   tract_svi['bv_lm_svi_pval'] = `Bivariate Local Morans I p-value`
```

---

In [ ]:
# Your code here

tract_svi['bv_lm_svi'] = `Bivariate Local Morans I result`
tract_svi['bv_lm_svi_pval'] = `Bivariate Local Morans I p-value`

tract_svi

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert 'bv_lm_svi' in tract_svi.columns
assert 'bv_lm_svi_pval' in tract_svi.columns
assert tract_svi.at[10, 'bv_lm_svi'] == 4
assert tract_svi.at[10, 'bv_lm_svi_pval'] == 0.103

print("Success!")

Upon the completion of the exercise 10, the following code will give you the same map as shown below. 

<div>
<img src="./image/svi_bv_local_moran.png" width="300"/>
</div>



In [ ]:
# Enter results of Bivariate LISA into each census region
lm_dict = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

for idx, row in tract_svi.iterrows():
    if row['bv_lm_svi_pval'] < 0.05:
        tract_svi.loc[idx, f'LISA_CLS'] = lm_dict[row['bv_lm_svi']]
    else:
        tract_svi.loc[idx, f'LISA_CLS'] = 'NA'

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for key,val in lisa_color.items():
    tract_svi.loc[tract_svi['LISA_CLS'] == key].plot(ax=ax, color=val)

ax.set_axis_off()
plt.show()

# 5. K-Means Clustering

K-means clustering is a type of unsupervised learning, which is used when you have unlabeled data. This appraoch can find out the characteristics of regions such as a group of regions has high value for certain SVI data while having low value for other SVI data.

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

def determine_number_of_cluster(array):
    km_cost = []  # Sum of squared distances of samples to their closest cluster center.
    distortions = []  # the average of the squared distances from the cluster centers of the respective clusters.Typically, the Euclidean distance metric is used.
    km_silhouette = {}

    for i in range(2, 11):
        KM = KMeans(n_clusters=i, max_iter=999, n_init = 99, random_state=17)
        KM.fit(array)

        # Calculate Silhouette Scores
        preds = KM.predict(array)
        silhouette = silhouette_score(array, preds)
        km_silhouette[i] = silhouette

    print(max(km_silhouette, key=km_silhouette.get))
        
    return km_silhouette


def kmeans_cluster(array, num_of_cluster):
    kmeans = KMeans(n_clusters=num_of_cluster, max_iter=999, n_init = 99, random_state=17)
    kmeans.fit(array)
    y_kmeans = kmeans.predict(array)
    
    cluster_df = pd.DataFrame({'cluster': y_kmeans}, index=array.index)
    cluster_df['cluster'] = cluster_df['cluster'].astype(str)
    
    return cluster_df

silhouette_scores = determine_number_of_cluster(tract_svi[['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']])
print({idx: round(val, 4) for idx, val in silhouette_scores.items()})


In [ ]:
# We select the second highest silhouette coefficient as the highest silhouette coefficient indicates only 2 groups. 
mi_copy = kmeans_cluster(tract_svi[['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']], 3)
tract_svi['cluster'] = mi_copy['cluster']
tract_svi['cluster'] = tract_svi['cluster'].astype(str)
tract_svi

In [ ]:
gs_kw = dict(width_ratios=[1, 1], height_ratios=[1, 1, 1])
fig, axd = plt.subplot_mosaic([['left', 'rightc0'], 
                               ['left', 'rightc1'], 
                               ['left', 'rightc2'],
                              ],
                              gridspec_kw=gs_kw, 
                              figsize=(15, 8),
                            #   constrained_layout=True
                             )
print(type(axd), axd)

tract_svi.plot('cluster', ax=axd['left'], legend=True, cmap='Set1')

for cls in [0, 1, 2]:
    temp_df = tract_svi.loc[tract_svi['cluster'] == str(cls), ['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']]
    axd[f'rightc{cls}'].boxplot(temp_df, showfliers=False)
    axd[f'rightc{cls}'].set_title(f'Cluster {cls}')
    

# DONE